# 04 — IPW & AIPW Estimation (Causal Effect of Late Delivery on Review Score)

**Goal.** Estimate the causal effect of *late delivery* (T=1) on **review_score** (Y) using:
- **IPW** (Inverse Probability Weighting) with stabilized weights
- **AIPW** (Doubly Robust)

**Inputs.**
- `../data/processed/model_df_with_ps.csv` produced by the propensity notebook:

**Outputs.**
- ATE (and ATT/ATC optionally) with **95% bootstrap CIs**
- Weight & overlap diagnostics
- Saved results in `../experiments/results_ate.csv`


In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import joblib
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

In [ ]:
# Load df with T, Y, and propensity scores
df = pd.read_csv("../data/processed/model_df_with_ps.csv")

# Load the EXACT SAME X used in Notebook 3
X = pd.read_csv("../data/processed/X_ps.csv")


print(df.shape, X.shape)
df.head(3)

(110770, 19) (110770, 13)


,late_delivery,review_score,order_purchase_dayofweek,order_purchase_hour,estimated_delivery_days,price,freight_value,product_volume_cm3,payment_value,payment_type,customer_state,customer_city,seller_state,seller_city,product_category_name,ps_logit,ps_gb,ipw,ipw_stabilized
0,0,4.0,0,10,15,29.99,8.72,1976.0,38.71,credit_card,SP,sao paulo,SP,maua,utilidades_domesticas,0.045035,0.042196,1.047159,0.967589
1,0,4.0,1,20,19,118.70,22.76,4693.0,141.46,boleto,BA,barreiras,SP,belo horizonte,perfumaria,0.204520,0.191663,1.257103,1.161581
2,0,5.0,2,8,26,159.90,19.22,9576.0,179.12,credit_card,GO,vianopolis,SP,guariba,automotivo,0.094467,0.106645,1.104322,1.020409


In [22]:
T = df["late_delivery"].astype(int).values
Y = df["review_score"].astype(float).values

# Prefer ps_logit if available
ps_col = "ps_logit" if "ps_logit" in df.columns else "ps_gb"
p = df[ps_col].clip(1e-6, 1-1e-6).values

# Use stabilized weights (already computed in Notebook 3)
W = df["ipw_stabilized"].values

IPW Estimators: ATE / ATT / ATC

We implement:
- **ATE (unstabilized)**
- **ATE (stabilized)**
- Optional **ATT** and **ATC**

Then we compute **95% bootstrap CIs** (adjust `B` if runtime is a concern).

In [18]:
rng = np.random.default_rng(9527)

def ipw_ate(T, Y, p):
    p = np.clip(p, 1e-6, 1-1e-6)
    tau = np.mean(T * Y / p) - np.mean((1 - T) * Y / (1 - p))
    return float(tau)

def ipw_att(T, Y, p):
    p = np.clip(p, 1e-6, 1-1e-6)
    mu1 = Y[T==1].mean()
    w_c = (p/(1-p))[T==0]
    mu0 = np.average(Y[T==0], weights=w_c)
    return float(mu1 - mu0)

def ipw_atc(T, Y, p):
    p = np.clip(p, 1e-6, 1-1e-6)
    mu1 = np.average(Y[T==1], weights=((1-p)/p)[T==1])
    mu0 = Y[T==0].mean()
    return float(mu1 - mu0)

def bootstrap_ci(estimator, B, T, Y, p):
    n = len(Y)
    vals = []
    rng = np.random.default_rng(123)
    for _ in range(B):
        idx = rng.integers(0, n, n)
        vals.append(estimator(T[idx], Y[idx], p[idx]))
    lo, hi = np.quantile(vals, [0.025, 0.975])
    return np.mean(vals), (lo, hi)

In [19]:
ate_ipw = ipw_ate(T, Y, p)
att_ipw = ipw_att(T, Y, p)
atc_ipw = ipw_atc(T, Y, p)

ate_boot, (lo_ate, hi_ate) = bootstrap_ci(ipw_ate, 300, T, Y, p)
att_boot, (lo_att, hi_att) = bootstrap_ci(ipw_att, 300, T, Y, p)
atc_boot, (lo_atc, hi_atc) = bootstrap_ci(ipw_atc, 300, T, Y, p)

print(f"IPW ATE: {ate_ipw:.4f}   (CI: {lo_ate:.4f}, {hi_ate:.4f})")
print(f"IPW ATT: {att_ipw:.4f}   (CI: {lo_att:.4f}, {hi_att:.4f})")
print(f"IPW ATC: {atc_ipw:.4f}   (CI: {lo_atc:.4f}, {hi_atc:.4f})")

IPW ATE: -1.5221   (CI: -1.6252, -1.4223)
IPW ATT: -1.6244   (CI: -1.6599, -1.5821)
IPW ATC: -1.6507   (CI: -1.7002, -1.5902)


## AIPW (Doubly Robust) Estimation

We fit two **outcome models**:
- $\hat\mu_1(x) = \mathbb{E}[Y \mid X=x, T=1]$
- $\hat\mu_0(x) = \mathbb{E}[Y \mid X=x, T=0]$

Then compute the AIPW estimator:
$$
\hat\tau_{AIPW}
= \frac{1}{n}\sum_i \Big[
\frac{T_i\{Y_i-\hat\mu_1(X_i)\}}{\hat e(X_i)}
-\frac{(1-T_i)\{Y_i-\hat\mu_0(X_i)\}}{1-\hat e(X_i)}
+\hat\mu_1(X_i)-\hat\mu_0(X_i)\Big].
$$

Notes:
- Uses **the same features X** (one-hot encoded via ColumnTransformer).
- Uses **GradientBoostingRegressor** (robust, few deps).
- We also compute **95% bootstrap CIs**.


In [24]:
# 1. Load and apply the saved preprocessing to ALL X
preprocess_ps = joblib.load("../models/preprocess_ps.pkl")
X_enc = preprocess_ps.transform(X)
if hasattr(X_enc, "toarray"):
    X_enc = X_enc.toarray()

# 2. Fit outcome models on encoded subsets
mu1 = GradientBoostingRegressor(random_state=9527)
mu0 = GradientBoostingRegressor(random_state=9527)

mu1.fit(X_enc[T==1], Y[T==1])
mu0.fit(X_enc[T==0], Y[T==0])

# 3. Predict for all
mu1_hat = mu1.predict(X_enc)
mu0_hat = mu0.predict(X_enc)


def aipw(T, Y, p, mu1, mu0):
    p = np.clip(p, 1e-6, 1-1e-6)
    part1 = T * (Y - mu1) / p
    part0 = (1 - T) * (Y - mu0) / (1 - p)
    tau = np.mean(part1 - part0 + (mu1 - mu0))
    return float(tau)

In [25]:
tau_aipw = aipw(T, Y, p, mu1_hat, mu0_hat)

# Bootstrap AIPW (fast version – reuse fitted models)
def aipw_bootstrap(T, Y, p, mu1_hat, mu0_hat, B=300):
    n=len(Y); vals=[]
    rng=np.random.default_rng(123)
    for _ in range(B):
        idx=rng.integers(0,n,n)
        vals.append(aipw(T[idx], Y[idx], p[idx], mu1_hat[idx], mu0_hat[idx]))
    return np.mean(vals), np.quantile(vals,[0.025,0.975])

aipw_mean,(lo,hi)=aipw_bootstrap(T,Y,p,mu1_hat,mu0_hat)

print(f"AIPW ATE: {tau_aipw:.4f}  (CI: {lo:.4f}, {hi:.4f})")

AIPW ATE: -1.6383  (CI: -1.6901, -1.5849)
